![DLI Header](images/DLI_Header.png)

# 聊天机器人
｜Chatbot

在此笔记本中，您将开始创建聊天机器人功能，创建一个能够保留对话历史记录的 AI 机器人。

## 学习目标
｜Learning Objectives

在完成此笔记本时，您将能够：
- 从我们的 LLaMA-2 模型创建聊天机器人功能，能够保留对话历史记录。

## 视频演练

执行下面的单元格以加载此笔记本的视频演练。

In [ ]:
 from IPython.display import HTML

video_url = "https://d36m44n9vdbmda.cloudfront.net/assets/s-fx-12-v1/v2/06-chatbot.mp4"

video_html = f"""
<video controls width="640" height="360">
    <source src="{video_url}" type="video/mp4">
    Your browser does not support the video tag.
</video>
"""

display(HTML(video_html))

## 创建 LLaMA-2 管道
｜Create LLaMA-2 Pipeline

In [ ]:
from transformers import pipeline
model = "TheBloke/Llama-2-13B-chat-GPTQ"
# model = "TheBloke/Llama-2-7B-chat-GPTQ"

llama_pipe = pipeline("text-generation", model=model, device_map="auto");

## 辅助函数和类
｜Helper Functions and Classes

在此笔记本中，我们将使用以下函数和类来支持我们与 LLM 的交互。现在可以随意浏览一下，因为在下面使用时会更详细地介绍。

### 生成模型响应
｜Generate Model Responses

In [ ]:
def generate(prompt, max_length=1024, pipe=llama_pipe, **kwargs):
    """
    Generates a response to the given prompt using a specified language model pipeline.

    This function takes a prompt and passes it to a language model pipeline, such as LLaMA, 
    to generate a text response. The function is designed to allow customization of the 
    generation process through various parameters and keyword arguments.

    Parameters:
    - prompt (str): The input text prompt to generate a response for.
    - max_length (int): The maximum length of the generated response. Default is 1024 tokens.
    - pipe (callable): The language model pipeline function used for generation. Default is llama_pipe.
    - **kwargs: Additional keyword arguments that are passed to the pipeline function.

    Returns:
    - str: The generated text response from the model, trimmed of leading and trailing whitespace.

    Example usage:
    ```
    prompt_text = "Explain the theory of relativity."
    response = generate(prompt_text, max_length=512, pipe=my_custom_pipeline, temperature=0.7)
    print(response)
    ```
    """

    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

### 构造提示，可选地使用系统上下文和/或示例
｜Costruct Prompt, Optionally With System Context and/or Examples

In [ ]:
def construct_prompt_with_context(main_prompt, system_context="", conversation_examples=[]):
    """
    Constructs a complete structured prompt for a language model, including optional system context and conversation examples.

    This function compiles a prompt that can be directly used for generating responses from a language model. 
    It creates a structured format that begins with an optional system context message, appends a series of conversational 
    examples as prior interactions, and ends with the main user prompt. If no system context or conversation examples are provided,
    it will return only the main prompt.

    Parameters:
    - main_prompt (str): The core question or statement for the language model to respond to.
    - system_context (str, optional): Additional context or information about the scenario or environment. Defaults to an empty string.
    - conversation_examples (list of tuples, optional): Prior exchanges provided as context, where each tuple contains a user message 
      and a corresponding agent response. Defaults to an empty list.

    Returns:
    - str: A string formatted as a complete prompt ready for language model input. If no system context or examples are provided, returns the main prompt.

    Example usage:
    ```
    main_prompt = "I'm looking to improve my dialogue writing skills for my next short story. Any suggestions?"
    system_context = "User is an aspiring author seeking to enhance dialogue writing techniques."
    conversation_examples = [
        ("How can dialogue contribute to character development?", "Dialogue should reveal character traits and show personal growth over the story arc."),
        ("What are some common pitfalls in writing dialogue?", "Avoid exposition dumps in dialogue and make sure each character's voice is distinct.")
    ]

    full_prompt = construct_prompt_with_context(main_prompt, system_context, conversation_examples)
    print(full_prompt)
    ```
    """
    
    # Return the main prompt if no system context or conversation examples are provided
    if not system_context and not conversation_examples:
        return main_prompt

    # Start with the initial part of the prompt including the system context, if provided
    full_prompt = f"<s>[INST] <<SYS>>{system_context}<</SYS>>\n" if system_context else "<s>[INST]\n"

    # Add each example from the conversation_examples to the prompt
    for user_msg, agent_response in conversation_examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    # Add the main user prompt at the end
    full_prompt += f"{main_prompt} [/INST]"

    return full_prompt

### LlamaChatbot 类
｜LlamaChatbot Class

In [ ]:
class LlamaChatbot:
    """
    A chatbot interface for generating conversational responses using the LLaMA language model.

    Attributes:
    - system_context (str): Contextual information to provide to the language model for all conversations.
    - conversation_history (list of tuples): Stores the history of the conversation, where each
      tuple contains a user message and the corresponding agent response.
    """

    def __init__(self, system_context):
        """
        Initializes a new instance of the LlamaChatbot class.

        Parameters:
        - system_context (str): A string that sets the initial context for the language model.
        """
        self.system_context = system_context
        self.conversation_history = []  # Initializes the conversation history

    def chat(self, user_msg):
        """
        Generates a response from the chatbot based on the user's message.

        This method constructs a prompt with the current system context and conversation history,
        sends it to the language model, and then stores the new user message and model's response
        in the conversation history.

        Parameters:
        - user_msg (str): The user's message to which the chatbot will respond.

        Returns:
        - str: The generated response from the chatbot.
        """
        # Generate the prompt using the conversation history and the new user message
        prompt = construct_prompt_with_context(user_msg, self.system_context, self.conversation_history)
        
        # Get the model's response
        agent_response = generate(prompt)

        # Store this interaction in the conversation history
        self.conversation_history.append((user_msg, agent_response))

        return agent_response

    def reset(self):
        """
        Resets the conversation history of the chatbot.

        This method clears the existing conversation history, effectively restarting the conversation.
        """
        # Clear conversation history
        self.conversation_history = []

## 没有对话记忆
｜No Conversation Memory

让我们与我们的 LLM 开始对话。我们将提供一个**系统上下文** ，即它应该是一个友好的聊天机器人，并且（天真地）鼓励它始终记住我们在对话中提供的姓名。

In [ ]:
system_context = """
You are a friendly chatbot always eager to help and engage in meaningful conversation. \
You always remember the details of our previous conversations, \
especially if a user gives them their name.
"""

prompt = "Hello my name is Star. Nice to meet you!"

print(generate(construct_prompt_with_context(prompt, system_context)))

---

该模型肯定似乎渴望表明它记得我们是谁。让我们看看当我们真正对其姓名保留能力进行测试时会发生什么。

In [ ]:
system_context = """
You are a friendly chatbot always eager to help and engage in meaningful conversation. \
You always remember the details of our previous conversations, \
especially if a user gives them their name.
"""

prompt = "Can you remind me what my name is?"

print(generate(construct_prompt_with_context(prompt, system_context)))

---

它可能并不令人意外，该模型并不记得我们的姓名，因为尽管它向我们展示了自己，但我们没有提供任何能力来记住之前对话交换中的任何细节。该模型似乎坚持认为我们的姓名是“Emily”，这显然是不正确的。当模型生成虚构的响应时，通常充满信心，我们称之为**幻觉（hallucination）** 。

## 创建对话记忆
｜Create Conversation Memory

为了创建聊天机器人体验，以便模型能够保留先前交换的信息，我们将使用一个`LlamaChatbot`类（如上所定义）。以下是我们类定义的`help`输出。

In [ ]:
help(LlamaChatbot)

---

与我们的直接目标最相关的是创建`conversation_history`列表，我们将在每次调用`chat`方法时追加到该列表。我们将重用之前笔记本中的一些相同逻辑，特别是利用 LLaMA-2 的**提示模板（prompt template）** ，以便每次用户/模型交互都正确格式化，然后在用户和模型之间后续交换的提示之前添加。

可以准确地说，在每次交互中，我们都在执行**少样本学习（few-shot learning）** ，其中指示性示例仅仅是先前的交互。

让我们看看它的实际效果。

In [ ]:
system_context = """
You are a friendly chatbot always eager to help and engage in meaningful conversation. You are always kind \
but also repectful and professional.
"""

chatbot = LlamaChatbot(system_context)

In [ ]:
print(chatbot.chat("Hi, my name is Star. Nice to meet you!"))

---

到目前为止一切顺利。现在让我们看看模型是否能够“回忆”起我们的姓名。

In [ ]:
print(chatbot.chat("Can you remind me what my name is?"))

---

成功！让我们看一下模型的对话历史记录。

In [ ]:
chatbot.conversation_history

---

鉴于`conversation_history`被添加到每个新提示之前，因此模型能够根据先前的交换生成响应是有道理的。

`reset`方法将清除`conversation_history`。

In [ ]:
chatbot.reset()

In [ ]:
print(chatbot.chat("Can you remind me what my name is?"))

---

现在，模型无法“回忆”起我们先前交换的详细信息也就不足为奇了。

## 练习：任务跟踪器
｜Exercise: Task Tracker

创建一个助手，可以跟踪您今天需要完成的事情。它应该能够根据您的对话添加和删除列表中的内容，并且在任何给定时间准确地提醒您还有哪些事情需要做。

如果您遇到困难，请查看下面的解决方案。

### 您的工作在此处

## 解决方案

In [ ]:
system_context = """
You are an assistant that help me keep track of what I need to do. You keep track of tasks that I \
provide you, and when asked remind me of what I have left to do.
"""

chatbot = LlamaChatbot(system_context)

In [ ]:
print(chatbot.chat("I need to do the following things today: eat breakfast, eat lunch, eat dinner, go to work, exercise, and clean the house"))

In [ ]:
print(chatbot.chat("Yes, I also need to hang out with friends."))

In [ ]:
print(chatbot.chat("Okay, I'm done eating breakfast and exercising."))

In [ ]:
print(chatbot.chat("I've eaten lunch. Sometime today I need to call the bike shop."))

In [ ]:
print(chatbot.chat("I finished work, hung out with friends, cleaned the house, and called the bike shop."))

In [ ]:
print(chatbot.chat("I just ate dinner. Now I just need to go to sleep."))

In [ ]:
chatbot.reset()

## 关键概念回顾

本笔记本中介绍了以下关键概念：

- **幻觉（Hallucination）：** 当模型生成通常带有某种表达的自信的虚假或不准确的响应时。

## 可选的高级练习

**注意：** 在下一个笔记本中，我们将发现模型能够存储的对话量存在限制，在超过限制后，事情开始出错。考虑到这一点，并且在建议其他实验之前，如果您发现模型开始仅产生空响应，请继续下一节，以便您了解这是怎么回事。

如果您想超越课程的要求，以下是一些供您尝试的额外开放式练习。

### 使用 7B 模型
｜Use the 7B Model

在笔记本的顶部，在重新启动内核（请参见下面的单元格）之后，取消注释并使用 7B 模型而不是我们演示的 13B 模型。尝试在使用较小（较弱）模型的情况下获得令人满意的结果。

### 创建一个助手机器人
｜Make a Helper Bot

创建一个机器人，人们在度过艰难的一天时可以使用它，它会给予鼓励、赞扬和同理心，并知道何时响应以及何时询问有关用户正在发生的事情的更多问题。

## 重新启动内核
｜Restart the Kernel

为了释放下一个笔记本的 GPU 内存，请运行以下单元格以重新启动内核。

In [ ]:
from IPython import get_ipython

get_ipython().kernel.do_shutdown(restart=True)

# [5LOI DEEP LEARNING INSTITUE](https://5loi.com)

### [AI COMMUNITY](https://www.theforage.cn/community)

### [5LOI](https://5loi.com/about_loi)

![DLI Header](images/DLI_Header.png)